Loading selected 16 articles into dict of 16 strings to later turn them into documents

In [2]:
import os
import dotenv
dotenv.load_dotenv()

True

In [3]:
import json
import pandas as pd
from collections import defaultdict

wanted = {}
with open('./data/stanford-question-answering-dataset/dev-v1.1.json', 'r', encoding='utf-8') as f:
    squad_data = json.load(f)
all = squad_data['data']
for i, article in enumerate(all):
    if i==4 or i==19 :
        wanted[article['title']] = article['paragraphs']
with open('./data/stanford-question-answering-dataset/train-v1.1.json', 'r', encoding='utf-8') as f:
    squad_data = json.load(f)
all = squad_data['data']
for i, article in enumerate(all):
    if i in {30, 62, 104, 124, 149, 157, 177, 229, 293, 295, 330, 333, 389, 393} :
        wanted[article['title']] = article['paragraphs']

article_dict = {}
context_dict = defaultdict(str)
rows = []
article_id=0
context_id=0
for title, contexts in wanted.items():
    article_dict[article_id] = title
    for i, element in enumerate(contexts):
        # for getting paragraphs alone
        # context_dict[context_id]=element.get('context')
        # for getting whole article text
        context_dict[article_id]+=element.get('context')
        qas = element.get('qas', [])
        for qa in qas:
            row = {
                'article_id': article_id,
                'context_id': context_id,
                'question_id': qa.get('id'),
                'question': qa.get('question'),
                'answers(list of dict)': qa.get('answers') # leave this for later i guess
            }
            rows.append(row)
        context_id+=1
    article_id+=1

result_df = pd.DataFrame(rows)


In [4]:
textovi = dict(context_dict)

In [5]:
import langchain_text_splitters as lts
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [6]:
splitter = lts.RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.create_documents(textovi.values())
splits = splitter.split_documents(docs)

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embed = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [8]:
vectorstore = Chroma.from_documents(documents=splits,embedding=embed)
retriever = vectorstore.as_retriever()

Kod iznad se prvi put izvrsavao skoro 7 minuta

In [14]:
givendocs = retriever.invoke("What branch of theoretical computer science de")
givendocs

[Document(metadata={}, page_content='Computational complexity theory is a branch of the theory of computation in theoretical computer science that focuses on classifying computational problems according to their inherent difficulty, and relating those classes to each other. A computational problem is understood to be a task that is in principle amenable to being solved by a computer, which is equivalent to stating that the problem may be solved by mechanical application of mathematical steps, such as an algorithm.A problem is regarded as inherently difficult if its solution requires significant resources, whatever the algorithm used. The theory formalizes this intuition, by introducing mathematical models of computation to study these problems and quantifying the amount of resources needed to solve them, such as time and storage. Other complexity measures are also used, such as the amount of communication (used in communication complexity), the number of gates in a circuit (used in cir

[Document(metadata={}, page_content='the original 128K Mac, was an all-in-one computer. Its translucent plastic case, originally Bondi blue and later various additional colors, is considered an industrial design landmark of the late 1990s. The iMac did away with most of Apple\'s standard (and usually proprietary) connections, such as SCSI and ADB, in favor of two USB ports. It replaced a floppy disk drive with a CD-ROM drive for installing software, but was incapable of writing to CDs or other media without external third-party hardware. The iMac proved to be phenomenally successful, with 800,000 units sold in 139 days. It made the company an annual profit of US$309 million, Apple\'s first profitable year since Michael Spindler took over as CEO in 1995. This aesthetic was applied to the Power Macintosh and later the iBook, Apple\'s first consumer-level laptop computer, filling the missing quadrant of Apple\'s "four-square product matrix" (desktop and portable products for both consumer